# Configure Cluster Module Params

This notebook should be used as a test for ensuring correct cluster parameters before cluster processing.
Cells marked with <font color='red'>SET PARAMETERS</font> contain crucial variables that need to be set according to your specific experimental setup and data organization.
Please review and modify these variables as needed before proceeding with the analysis.

## <font color='red'>SET PARAMETERS</font>

### Fixed parameters for cluster module

- `CONFIG_FILE_PATH`: Path to a Brieflow config file used during processing. Absolute or relative to where workflows are run from.

In [ ]:
CONFIG_FILE_PATH = "config/config.yml"

In [ ]:
from pathlib import Path

import yaml
import pandas as pd
from matplotlib import pyplot as plt

from lib.cluster.cluster_eval import plot_cell_histogram
from lib.cluster.generate_dataset import (
    clean_and_validate,
    split_channels,
    remove_low_number_genes,
    remove_missing_features,
)
from lib.cluster.phate_leiden_clustering import (
    select_features,
    normalize_to_controls,
    perform_pca_analysis,
    phate_leiden_pipeline,
    dimensionality_reduction,
    merge_phate_uniprot,
)
from lib.cluster.benchmark_clusters import (
    create_cluster_gene_table,
    analyze_differential_features,
    process_interactions,
)
from lib.cluster.scrape_benchmarks import (
    get_uniprot_data,
    get_string_data,
    get_corum_data,
)
from lib.shared.configuration_utils import CONFIG_FILE_HEADER

In [ ]:
# load config file and determine root path
with open(CONFIG_FILE_PATH, "r") as config_file:
    config = yaml.safe_load(config_file)
    ROOT_FP = Path(config["all"]["root_fp"])

## <font color='red'>SET PARAMETERS</font>

### Cluster processing

- `CHANNEL_COMBOS`: Combinations of channels to use during cluster process.
- `DATASET_TYPES`: Dataset types to use during cluster process; created during the aggregate process.
- `MIN_CELL_CUTOFFS`: Minimum cells for each gene to be used in clusetering analysis. More cells per gene increases confidence, but some dataset types (ex mitotic) may have an inherently low number of cells for a particular perturbation.

In [ ]:
# parameters for all cluster processing
CHANNEL_COMBOS = None
DATASET_TYPES = None
MIN_CELL_CUTOFFS = None

In [ ]:
for dataset_type, min_cell_cutoff in MIN_CELL_CUTOFFS.items():
    # load and clean relevant dataset
    gene_data_fp = ROOT_FP / "aggregate" / "tsvs" / f"{dataset_type}_gene_data.tsv"
    gene_data = pd.read_csv(gene_data_fp, sep="\t")
    cleaned_data = clean_and_validate(gene_data)

    # show cell count distribution
    print(f"Cell count distribution for: {dataset_type}")
    plot_cell_histogram(cleaned_data, min_cell_cutoff)
    plt.show()

## <font color='red'>SET PARAMETERS</font>

### Cluster testing

- `TEST_CHANNEL_COMBO`: Channel combination to use for testing cluster process. Should be smallest or most relevant channel combination.
- `TEST_DATASET_TYPE`: Dataset type to use for testing cluster process. Should be smallest or most relevant dataset type.

In [ ]:
# parameters for testing cluster processing
TEST_CHANNEL_COMBO = None
TEST_DATASET_TYPE = None

In [ ]:
# load gene data based on dataset we are processing
gene_data_fp = ROOT_FP / "aggregate" / "tsvs" / f"{TEST_DATASET_TYPE}_gene_data.tsv"
gene_data = pd.read_csv(gene_data_fp, sep="\t")

# clean and validate gene data
validated_data = clean_and_validate(gene_data)

# filter dataset for channels of interest
channel_filtered_data = split_channels(
    validated_data,
    TEST_CHANNEL_COMBO,
    config["phenotype"]["channel_names"],
)

# clean low number genes and missing features
cleaned_data = remove_low_number_genes(
    channel_filtered_data, MIN_CELL_CUTOFFS[TEST_DATASET_TYPE]
)
cleaned_data = remove_missing_features(cleaned_data)

cleaned_data

## <font color='red'>SET PARAMETERS</font>

### Feature selection

- `CORRELATION_THRESHOLD`: Features with a correlation (to another feature) above this threshold are iteratively removed to get "unique" features.
- `VARIANCE_THRESHOLD`: Features with variance below this threshold are removed.
- `MIN_UNIQUE_VALUES`: Features with a number of unique values below this threshold are removed.

In [ ]:
# parameters used for feature selection
CORRELATION_THRESHOLD = None
VARIANCE_THRESHOLD = None
MIN_UNIQUE_VALUES = None

In [ ]:
# select features
filtered_data, removed_features = select_features(
    cleaned_data,
    correlation_threshold=CORRELATION_THRESHOLD,
    variance_threshold=VARIANCE_THRESHOLD,
    min_unique_values=MIN_UNIQUE_VALUES,
)

filtered_data

## <font color='red'>SET PARAMETERS</font>

### Phate/Leiden Clustering

- `CUM_VAR_THRESHOLD`: Minimum cumulative variance that needs to be explained by selected PCA features. Higher variance = more features.
- `LEIDEN_RESOLUTION`: Resolution for Leiden clustering. 

In [ ]:
CUM_VAR_THRESHOLD = None
LEIDEN_RESOLUTION = None

In [ ]:
CONTROL_PREFIX = config["aggregate"]["control_prefix"]
POPULATION_FEATURE = config["aggregate"]["population_feature"]

# normalize filtered data
normalized_data = normalize_to_controls(filtered_data, CONTROL_PREFIX)

# threshold data with pca
pca_thresholded_data, n_components, pca, fig = perform_pca_analysis(
    normalized_data, CUM_VAR_THRESHOLD
)
fig.show()

# perform phate leiden clustering
phate_leiden_clustering = phate_leiden_pipeline(
    pca_thresholded_data, resolution=LEIDEN_RESOLUTION
)

# create plot with phate leiden clustering
dimensionality_reduction(
    phate_leiden_clustering,
    x="PHATE_0",
    y="PHATE_1",
    control_query=f'{POPULATION_FEATURE}.str.startswith("{CONTROL_PREFIX}")',
    control_color="lightgray",
    control_legend=True,
    label_query=f'~{POPULATION_FEATURE}.str.startswith("{CONTROL_PREFIX}")',
    label_hue="cluster",
    label_palette="husl",
    s=25,
    hide_axes=False,
    label_legend=False,
    legend_kwargs={"loc": "center left", "bbox_to_anchor": (1, 0.5)},
)

## <font color='red'>SET PARAMETERS</font>

### Cluster Analysis

- `UNIPROT_DATA_FP`: Path to save and access UniProt data with gene descriptions.
- `STRING_DATA_FP`: Path to save and access STRING data with gene pairs.
- `CORUM_DATA_FP`: Path to save and access CORUM data with gene complexes.

In [ ]:
UNIPROT_DATA_FP = "config/benchmark_clusters/uniprot_data.tsv"
STRING_DATA_FP = "config/benchmark_clusters/string_data.tsv"
CORUM_DATA_FP = "config/benchmark_clusters/corum_data.tsv"

In [ ]:
# get and save uniprot data
uniprot_data = get_uniprot_data()
uniprot_data.to_csv(UNIPROT_DATA_FP, sep="\t", index=False)
display(uniprot_data)

# get and save string data
string_data = get_string_data()
string_data.to_csv(STRING_DATA_FP, sep="\t", index=False)
display(string_data)

# get and save corum data
corum_data = get_corum_data()
corum_data.to_csv(CORUM_DATA_FP, sep="\t", index=False)
display(corum_data)

In [ ]:
# load cluster data with uniprot annotations
phate_leiden_uniprot = merge_phate_uniprot(phate_leiden_clustering, UNIPROT_DATA_FP)

# create cluster gene table
cluster_gene_table = create_cluster_gene_table(
    phate_leiden_uniprot,
    columns_to_combine=[POPULATION_FEATURE, "STRING"],
)

# analyze differential features
cluster_gene_table, diff_results = analyze_differential_features(
    cluster_gene_table, cleaned_data
)

# process interactions and get enrichment results
cluster_gene_table, global_metrics = process_interactions(
    cluster_gene_table, STRING_DATA_FP, CORUM_DATA_FP
)

# show cluster analysis results
display(cluster_gene_table)
display(global_metrics)

## Add cluster parameters to config file

In [ ]:
# Add cluster section
config["cluster"] = {
    "channel_combos": CHANNEL_COMBOS,
    "dataset_types": DATASET_TYPES,
    "min_cell_cutoffs": MIN_CELL_CUTOFFS,
    "correlation_threshold": CORRELATION_THRESHOLD,
    "variance_threshold": VARIANCE_THRESHOLD,
    "min_unique_values": MIN_UNIQUE_VALUES,
    "cum_var_threshold": CUM_VAR_THRESHOLD,
    "leiden_resolution": LEIDEN_RESOLUTION,
    "uniprot_data_fp": UNIPROT_DATA_FP,
    "string_data_fp": STRING_DATA_FP,
    "corum_data_fp": CORUM_DATA_FP,
}

# Write the updated configuration
with open(CONFIG_FILE_PATH, "w") as config_file:
    # Write the introductory comments
    config_file.write(CONFIG_FILE_HEADER)

    # Dump the updated YAML structure, keeping markdown comments for sections
    yaml.dump(config, config_file, default_flow_style=False)